<a href="https://colab.research.google.com/github/lhmtri18/analysis_ml_dl_python/blob/master/order_brush_shopee_code_league.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Order brush detection- Shopee Code league


Code is written by Minh Tri, University of Science, VietNam

https://www.kaggle.com/c/students-2-shopee-code-league-order-brushing/overview/examples


In [0]:
import pandas as pd
order_brush= pd.read_csv('/order_brush_order.csv')#/order_brush_order.csv
order_brush

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [0]:
order_brush.shape#222750,4
len(order_brush['shopid'].unique())#18770 shopid
len(order_brush['userid'].unique())#93883
order_brush.dtypes

orderid        int64
shopid         int64
userid         int64
event_time    object
dtype: object

Check missing value

In [0]:
def check_null(x):
  return any(x.isnull())
order_brush.apply(check_null)

orderid       False
shopid        False
userid        False
event_time    False
dtype: bool

Ops, no missing values

Time datatype is object, we need to change it to datetime datatype

In [0]:
from datetime import datetime
replica_order_brush= order_brush.copy()#make a replica, just in case lost
order_brush['event_time']=[datetime.strptime(x,'%Y-%m-%d %H:%M:%S')for x in order_brush['event_time']]
order_brush['event_time'].head()

0   2019-12-27 00:23:03
1   2019-12-27 11:54:20
2   2019-12-27 13:22:35
3   2019-12-27 13:01:00
4   2019-12-27 11:37:55
Name: event_time, dtype: datetime64[ns]

In [0]:
order_brush.shape

(222750, 4)

Ok, event_time is in datetime datatype

In [0]:
unique_shopid= order_brush.shopid.unique()
from pandas.tseries.offsets import Day,Hour
#new_order_brush= pd.DataFrame(None)
res= dict()
for shop_id in unique_shopid:
    temp_shop= order_brush[order_brush['shopid']==shop_id]#create table with one shopid
    temp_shop= temp_shop.sort_values(by='event_time',ascending=True)#sort table by time
    max_rate = 0
    list_userid = set()
    flag = False
    
    #print('temp_shop: ',temp_shop)

    #temp_shop= temp_shop.reindex(range(len(temp_shop)))
    for i in temp_shop.index: #go through one by one order
        lowerbound= temp_shop.loc[i,'event_time']
        upperbound= temp_shop.loc[i,'event_time'] + Hour()#plus 1 hour
        interval_time = temp_shop[(temp_shop['event_time'] >= lowerbound) & (temp_shop['event_time'] <upperbound)]
        #print('interval_time: ',interval_time)
        concentrate_rate = len(interval_time)/len(interval_time.userid.unique())#rate for this interval time
        #print('len_interval_time: ',len(interval_time))
        #print('len_userid: ',len(interval_time.userid.unique()))
        #print('concentrate_rate',concentrate_rate)
        if concentrate_rate>=3:#positive
            #print('shop id: ',shop_id)
            #print('concentrate_rate',concentrate_rate)
            flag = True
            order_by_userid= interval_time.groupby('userid')['orderid'].count()#group by userid for each time
            rate = pd.Series(order_by_userid / len(interval_time),index= order_by_userid.index)#rate each user
            max_cur_rate= max(rate)#max rate user
            #print('rate: ',rate)
            if max_cur_rate>max_rate:
                list_userid.clear()#clear all to update
                list_userid = set(order_by_userid[rate==max_cur_rate].index)#update new element
                #print('list_userid: ',list_userid)
                max_rate= max_cur_rate#update new rate
            elif max_cur_rate==max_rate:
                list_userid = list_userid|set(order_by_userid[rate==max_cur_rate].index)#update new element
                #print('list_userid: ',list_userid)
    if flag==True:
        res[shop_id]= list_userid
    else:
        res[shop_id] = '0'

#new_order_brush=pd.concat([new_order_brush,temp_shop])


In [0]:
for key,value in res.items():
  print(key,'  userid: ',value)

Streaming output truncated to the last 5000 lines.
183475983   userid:  0
208847574   userid:  0
189709678   userid:  0
199328999   userid:  0
182891695   userid:  0
186968762   userid:  0
190693708   userid:  0
14583474   userid:  0
208810209   userid:  0
25745311   userid:  0
186181028   userid:  0
163101297   userid:  0
29652577   userid:  0
173300715   userid:  0
203545757   userid:  0
120384767   userid:  0
162582771   userid:  0
206464158   userid:  0
191974584   userid:  0
173485445   userid:  0
151698344   userid:  0
182863255   userid:  0
160352250   userid:  0
170372511   userid:  0
199320334   userid:  0
163339479   userid:  0
118974943   userid:  0
59474096   userid:  0
182856600   userid:  0
182261551   userid:  0
42602990   userid:  0
203718335   userid:  0
170364649   userid:  0
191577883   userid:  0
130119388   userid:  0
201522855   userid:  0
206135587   userid:  0
133304660   userid:  0
159313616   userid:  0
34502659   userid:  0
190759944   userid:  0
14670331   u

In [0]:
len(res)

18770

In [0]:
for key,value in res.items():
  if value!=0:
    value= sorted(value)
    print(key,'  userid: ',value)

Streaming output truncated to the last 5000 lines.
183475983   userid:  ['0']
208847574   userid:  ['0']
189709678   userid:  ['0']
199328999   userid:  ['0']
182891695   userid:  ['0']
186968762   userid:  ['0']
190693708   userid:  ['0']
14583474   userid:  ['0']
208810209   userid:  ['0']
25745311   userid:  ['0']
186181028   userid:  ['0']
163101297   userid:  ['0']
29652577   userid:  ['0']
173300715   userid:  ['0']
203545757   userid:  ['0']
120384767   userid:  ['0']
162582771   userid:  ['0']
206464158   userid:  ['0']
191974584   userid:  ['0']
173485445   userid:  ['0']
151698344   userid:  ['0']
182863255   userid:  ['0']
160352250   userid:  ['0']
170372511   userid:  ['0']
199320334   userid:  ['0']
163339479   userid:  ['0']
118974943   userid:  ['0']
59474096   userid:  ['0']
182856600   userid:  ['0']
182261551   userid:  ['0']
42602990   userid:  ['0']
203718335   userid:  ['0']
170364649   userid:  ['0']
191577883   userid:  ['0']
130119388   userid:  ['0']
201522855

In [0]:
data= {'shopid':list(res.keys()),'userid':list(res.values())}
final_res= pd.DataFrame(data)
final_res.shape

(18770, 2)

In [0]:
final_res[final_res['userid']!='0']

,shopid,userid
34,1175477,{122277324}
70,66861410,{213502289}
72,8715449,{9753706}
140,58543771,{61893096}
141,156883302,"{76102350, 188025647}"
...,...,...
12754,78837196,{78903959}
12954,189544563,{799445}
13958,64369518,{172591713}
14302,83234229,{181942741}


In [0]:
final_res.head()

,shopid,userid
0,93950878,0
1,156423439,0
2,173699291,0
3,63674025,0
4,127249066,0


In [0]:
final_res.dtypes

shopid     int64
userid    object
dtype: object

In [0]:
userid_str= final_res.loc[:,'userid']
def join_res(x):
  x = [str(i) for i in x]
  if x!='0':
    x = '&'.join(x)
  return x 
userid_str= userid_str.apply(join_res)
#final_res[positive].loc[:,'userid']= '&'.join(final_res[positive].loc[:,'userid'])

In [0]:
final_res['userid']= userid_str
final_res.loc[0,:]

shopid    93950878
userid           0
Name: 0, dtype: object

In [0]:
final_res[final_res['userid']!='0']

,shopid,userid
34,1175477,122277324
70,66861410,213502289
72,8715449,9753706
140,58543771,61893096
141,156883302,76102350&188025647
...,...,...
12754,78837196,78903959
12954,189544563,799445
13958,64369518,172591713
14302,83234229,181942741


In [0]:
final_res.shape

(18770, 2)

In [0]:
final_res.to_csv('final_res.csv',index=False)

Finally, we got 88% correction rate from Kaggle submission

This code take long time to run, we should optimize it later